In [ ]:
## Author: Bin Xu
## Created on: 10/15/2022, The University of Oklahoma, Norman, OK, USA
## Please cite our original paper if you utilize this code in your publication.

## Code sections starting with a full line of "#" requires your attention to comment out and uncomment/
## based on your interested parameters
########################################################################################
import sys
sys.path.append('../src')
import os
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib notebook
import FASTSim_BX_EV_Tesla_TMS
import time
t0=time.time()

## parametric setup
car_list=['model3','modelS']
Cy_list=['udds','hwfet']
AC_switch_list=['on','off']

########################################################################################
## please only uncomment one of following three lines based on your interest study for ambient T, cab T or ess T.
T_amb_list=[30,35,40]; Var_len=len(T_amb_list)
# T_cab_list=[22,25,28]; Var_len=len(T_cab_list)
# T_ess_list=[22,25,28]; Var_len=len(T_ess_list)

## default temperature setup (might be changed in the for loop)
T_amb_ref=35
T_cab_ref=25 
T_ess_ref=25   

## four arrays to store range and energy consumption rate data
mi_3_on  = np.zeros([Var_len,4])
mi_3_off = np.zeros([Var_len,4])
mi_S_on  = np.zeros([Var_len,4])
mi_S_off = np.zeros([Var_len,4])

########################################################################################
## main loop (please only uncomment one of the i3 for loop for amb T, cab T, and ess T)
for i1 in range(len(car_list)):
    for i2 in range(len(AC_switch_list)):
        for i3 in range(len(T_amb_list)):
#         for i3 in range(len(T_cab_list)):
#         for i3 in range(len(T_ess_list)):
            for i4 in range(len(Cy_list)):           
                car=car_list[i1]
                AC_switch=AC_switch_list[i2]
                Cy=Cy_list[i4]
                
                ########################################################################################
                ## (please only uncomment one of the i3 for loop for amb T, cab T, and ess T)
                T_amb_ref=T_amb_list[i3]
#                 T_cab_ref=T_cab_list[i3]
#                 T_ess_ref=T_ess_list[i3] 

                print(i1,i2,i3,i4,'Sim(min):',round((time.time()-t0)/60,1),'Tesla',car,'AC:',AC_switch,Cy,'T amb:',T_amb_ref,'T cab',T_cab_ref,'T ess',T_ess_ref)
                ## Load Drive Cycle
                if Cy=='udds':
                    cyc = FASTSim_BX_EV_Tesla_TMS.get_standard_cycle("udds")
                if Cy=='hwfet':
                    cyc = FASTSim_BX_EV_Tesla_TMS.get_standard_cycle("hwfet")
                # cyc = FASTSim_BX_EV_model3_TMS.get_standard_cycle("wltp")

                ## choose Powertrain Model
                # A vehicle database in CSV format is required to be in the working directory where FASTSim is running 
                # (i.e. the same directory as this notebook). The "get_veh" function selects the appropriate vehicle
                # attributes from the database and contructs the powertrain model (engine efficiency map, etc.). 
                # An integer value corresponds to each vehicle in the database. To add a new vehicle, simply 
                # populate a new row to the vehicle database CSV.
                if car=='modelS':
                    veh_num=22 # 2016 tesla model s
                if car=='model3':
                    veh_num=23 # 2022 tesla model 3

                veh = FASTSim_BX_EV_Tesla_TMS.get_veh(veh_num)
                print(veh['vehPtType'],'/',veh['name'])

                ## assign the setup values to the veh variable
                veh['car']=car
                veh['Cy']=Cy
                veh['AC_switch']=AC_switch
                veh['T_amb_ref']=T_amb_ref
                veh['T_cab_ref']=T_cab_ref
                veh['T_ess_ref']=T_ess_ref

                ## run the simulation
                # Run FASTSim
                # The "sim_drive" function takes the drive cycle and vehicle models defined above as inputs. The output is a dictionary of time series and scalar values described the simulation results. Typically of interest is the "gge" key, which is an array of time series energy consumption data at each time step in the drive cycle. Additionally, to add a result from the simulator to the output dictionary, a user can modify the sim_drive_sub function source code to include the desired field.
                # If running FASTSim in batch over many drive cycles, the output from "sim_drive" can be written to files or database for batch post-processing.
                output = FASTSim_BX_EV_Tesla_TMS.sim_drive(cyc, veh)

                ## Extracting Results
                df = pd.DataFrame.from_dict(output)[['soc','fcKwOutAch','essKwOutAch',\
                                                     'mcMechKwOutAch','auxInKw','mcElecKwInAch','mpgge',\
                                                     'transKwInAch','mcMechKw4ForcedFc','canPowerAllElectrically',\
                                                     'fuelKg','fuelKgAch','mphAch','distance_mi','cycDragKw','distMiles',\
                                                    'cycRrKw','cycRegenBrakeKw','cycAccelKw','rfgPcompKw','evapPfanairKw']]

                df['speed'] = cyc['cycMps']
                df['soc']=df['soc']*100                
                mi=df['distance_mi'].iloc[-1]*100/(df['soc'].iloc[0]-df['soc'].iloc[-1])
                Epmi=veh['maxEssKwh']/mi*1e3
                print('Range(mi):',round(mi),'\nEnergy(Wh/mi):',round(Epmi))                
                
                ## assign range and energy consumption rate value in array for print purposes
                if i1==0:
                    if i2==0:
                        mi_3_on[i3,i4*2]     = round(mi,2)
                        mi_3_on[i3,i4*2+1]   = round(Epmi,2)
                    if i2==1:
                        mi_3_off[i3,i4*2]    = round(mi,2)
                        mi_3_off[i3,i4*2+1]  = round(Epmi,2)
                if i1==1:
                    if i2==0:
                        mi_S_on[i3,i4*2]     = round(mi,2)
                        mi_S_on[i3,i4*2+1]   = round(Epmi,2)
                    if i2==1:
                        mi_S_off[i3,i4*2]    = round(mi,2)
                        mi_S_off[i3,i4*2+1]  = round(Epmi,2)
                print('mi_3_on =\n',mi_3_on )
                print('mi_3_off=\n',mi_3_off)
                print('mi_S_on =\n',mi_S_on )
                print('mi_S_off=\n',mi_S_off)
                
                ## save range and energy consumption rate in dataframe
                df['fuelKg'].iloc[0]=mi
                df['fuelKg'].iloc[1]=Epmi
                
                ## save vehicle operating data for each parameter setup
                file_name='res_'+car+'_'+Cy+'_Tamb_'+str(T_amb_ref)+'_Tcab_'+str(T_cab_ref)+'_Tess_'+str(T_ess_ref)+'_'+AC_switch+'.csv'
                print('Saved file:',file_name)
                df.to_csv('data/'+file_name,index=False,encoding='utf-8')

print('Simulation ends')